In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/timesfm2.0_project

%cd /content/drive/MyDrive/timesfm2.0_project
!git clone https://github.com/google-research/timesfm
%cd timesfm

!pip install timesfm[torch] --upgrade
!pip install scikit-learn pandas numpy matplotlib tqdm joblib

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory // 1024**3
    print(f"GPU Memory: {gpu_memory} GB")

    if "T4" in torch.cuda.get_device_name(0):
        print("Detected T4 GPU - optimizing batch size for maximum utilization")
        optimal_batch_size = 128
        per_core_batch_size = 64
    else:
        optimal_batch_size = 64
        per_core_batch_size = 32
else:
    print("GPU not available - using CPU")
    optimal_batch_size = 16
    per_core_batch_size = 8

print(f"Optimal batch size: {optimal_batch_size}")
print(f"Per core batch size: {per_core_batch_size}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/timesfm2.0_project
fatal: destination path 'timesfm' already exists and is not an empty directory.
/content/drive/MyDrive/timesfm2.0_project/timesfm
PyTorch version: 2.6.0+cu124
CUDA available: True
Current CUDA device: 0
Device name: Tesla T4
GPU Memory: 14 GB
Detected T4 GPU - optimizing batch size for maximum utilization
Optimal batch size: 128
Per core batch size: 64


In [ ]:
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import timesfm
import os
import joblib
import gc
import random

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

print(f"PyTorch version: {torch.__version__}")
print(f"TimesFM library loaded successfully")
print(f"CUDA available: {torch.cuda.is_available()}")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.cuda.empty_cache()
    print("CUDA optimizations enabled")

print("Initializing TimesFM 2.0 model...")

backend_type = "gpu" if torch.cuda.is_available() else "cpu"

tfm = timesfm.TimesFm(
    hparams=timesfm.TimesFmHparams(
        backend=backend_type,
        per_core_batch_size=per_core_batch_size,
        horizon_len=1,
        context_len=2048,
        num_layers=50,
        use_positional_embedding=False,
    ),
    checkpoint=timesfm.TimesFmCheckpoint(
        huggingface_repo_id="google/timesfm-2.0-500m-pytorch"
    )
)

print(f"Successfully initialized TimesFM 2.0 model (500M parameters) using {backend_type}")
print(f"Model configuration: context_len={tfm.hparams.context_len}, per_core_batch_size={tfm.hparams.per_core_batch_size}")

data_path = "/content/drive/MyDrive/ERP Data/all_window_datasets_unscaled.npz"

if os.path.exists(data_path):
    print(f"Found data file at: {data_path}")
    data = np.load(data_path, allow_pickle=True)
    print("Data loaded successfully!")
    print(f"Data file contains keys: {list(data.keys())}")
else:
    raise FileNotFoundError(f"Data file not found at: {data_path}")

window_sizes = [5, 21, 252, 512]
results = {}

print("TimesFM 2.0 initialization completed!")


PyTorch version: 2.6.0+cu124
TimesFM library loaded successfully
CUDA available: True
Using device: cuda
CUDA optimizations enabled
Initializing TimesFM 2.0 model...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Successfully initialized TimesFM 2.0 model (500M parameters) using gpu
Model configuration: context_len=2048, per_core_batch_size=64
Found data file at: /content/drive/MyDrive/ERP Data/all_window_datasets_unscaled.npz
Data loaded successfully!
Data file contains keys: ['X_train_5', 'y_train_5', 'meta_train_5', 'market_caps_train_5', 'X_test_5', 'y_test_5', 'meta_test_5', 'market_caps_test_5', 'X_train_21', 'y_train_21', 'meta_train_21', 'market_caps_train_21', 'X_test_21', 'y_test_21', 'meta_test_21', 'market_caps_test_21', 'X_train_252', 'y_train_252', 'meta_train_252', 'market_caps_train_252', 'X_test_252', 'y_test_252', 'meta_test_252', 'market_caps_test_252', 'X_train_512', 'y_train_512', 'meta_train_512', 'market_caps_train_512', 'X_test_512', 'y_test_512', 'meta_test_512', 'market_caps_test_512']
TimesFM 2.0 initialization completed!


In [ ]:
def r2_zero(y_true, y_pred):
    """
    Compute zero-based R² (baseline is 0)
    y_true: true values array (N,)
    y_pred: predicted values array (N,)
    """
    rss = np.sum((y_true - y_pred)**2)
    tss = np.sum(y_true**2)
    return 1 - rss / tss

def calc_directional_metrics(y_true, y_pred, permnos=None):
    """
    Compute directional accuracy metrics.
    - Sign prediction at sample level
    - If grouped by stock, compute Overall, Up, Down for each stock and then average
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    if permnos is None:
        s_true = np.sign(y_true)
        s_pred = np.sign(y_pred)
        mask = s_true != 0
        s_true = s_true[mask]
        s_pred = s_pred[mask]

        overall_acc = np.mean(s_true == s_pred)

        up_mask = s_true > 0
        down_mask = s_true < 0
        up_acc = np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else 0
        down_acc = np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else 0

    else:
        df = pd.DataFrame({"permno": permnos, "yt": y_true, "yp": y_pred})
        overall_accs = []
        up_accs = []
        down_accs = []

        for _, g in df.groupby("permno"):
            s_true = np.sign(g["yt"].values)
            s_pred = np.sign(g["yp"].values)
            mask = s_true != 0
            s_true = s_true[mask]
            s_pred = s_pred[mask]
            if len(s_true) == 0:
                continue
            overall_accs.append(np.mean(s_true == s_pred))

            up_mask = s_true > 0
            down_mask = s_true < 0
            up_accs.append(np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else np.nan)
            down_accs.append(np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else np.nan)

        overall_acc = np.nanmean(overall_accs)
        up_acc = np.nanmean(up_accs)
        down_acc = np.nanmean(down_accs)

    return overall_acc, up_acc, down_acc

def calculate_metrics(y_true, y_pred, permnos=None, meta=None):
    """Compute evaluation metrics."""
    from sklearn.metrics import mean_squared_error, mean_absolute_error

    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)
    k = y_pred.shape[-1] if len(y_pred.shape) > 1 else 1

    r2 = r2_zero(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    dir_acc, up_acc, down_acc = calc_directional_metrics(y_true, y_pred, permnos)

    metrics = {
        "R²": r2,
        "MAE": mae,
        "MSE": mse,
        "Directional Accuracy": dir_acc,
        "Up_Directional_Acc": up_acc,
        "Down_Directional_Acc": down_acc
    }

    if meta is not None and "MKTCAP_PERCENTILE" in meta:
        top_mask = meta["MKTCAP_PERCENTILE"] >= 0.75
        bottom_mask = meta["MKTCAP_PERCENTILE"] <= 0.25

        if np.any(top_mask):
            yt_top = y_true[top_mask]
            yp_top = y_pred[top_mask]
            perm_top = permnos[top_mask] if permnos is not None else None
            r2_top = r2_zero(yt_top, yp_top)
            mae_top = mean_absolute_error(yt_top, yp_top)
            mse_top = mean_squared_error(yt_top, yp_top)
            dir_top, up_top, down_top = calc_directional_metrics(yt_top, yp_top, perm_top)
            metrics.update({
                "Top25_R2": r2_top,
                "Top25_MAE": mae_top,
                "Top25_MSE": mse_top,
                "Top25_Dir_Acc": dir_top,
                "Top25_Up_Acc": up_top,
                "Top25_Down_Acc": down_top
            })

        if np.any(bottom_mask):
            yt_bot = y_true[bottom_mask]
            yp_bot = y_pred[bottom_mask]
            perm_bot = permnos[bottom_mask] if permnos is not None else None
            r2_bot = r2_zero(yt_bot, yp_bot)
            mae_bot = mean_absolute_error(yt_bot, yp_bot)
            mse_bot = mean_squared_error(yt_bot, yp_bot)
            dir_bot, up_bot, down_bot = calc_directional_metrics(yt_bot, yp_bot, perm_bot)
            metrics.update({
                "Bottom25_R2": r2_bot,
                "Bottom25_MAE": mae_bot,
                "Bottom25_MSE": mse_bot,
                "Bottom25_Dir_Acc": dir_bot,
                "Bottom25_Up_Acc": up_bot,
                "Bottom25_Down_Acc": down_bot
            })

    return metrics

print("Evaluation functions defined successfully")


Evaluation functions defined successfully


In [ ]:
print("Starting TimesFM 2.0 prediction with T4-optimized batch processing...")

for window_size in window_sizes:
    print(f"\nProcessing Window Size: {window_size}")

    X_test = data[f"X_test_{window_size}"]
    y_test = data[f"y_test_{window_size}"]
    meta_test = pd.DataFrame(data[f"meta_test_{window_size}"].item())

    print(f"Test data shape: X_test={X_test.shape}, y_test={y_test.shape}")
    print(f"Positive ratio in test: {(y_test > 0).mean():.4f}")

    all_predictions = []

    total_samples = len(X_test)

    if torch.cuda.is_available():
        if window_size <= 21:
            batch_size = optimal_batch_size
        elif window_size <= 252:
            batch_size = optimal_batch_size // 2
        else:
            batch_size = optimal_batch_size // 4
    else:
        batch_size = 8

    print(f"Using batch size: {batch_size} for window size {window_size}")

    for i in tqdm(range(0, total_samples, batch_size), desc=f"Predicting w{window_size}"):
        batch_end = min(i + batch_size, total_samples)
        batch_X = X_test[i:batch_end]

        batch_predictions = []

        batch_sequences = []
        for j in range(len(batch_X)):
            sequence = batch_X[j].flatten()
            if len(sequence) > 2048:
                sequence = sequence[-2048:]
            batch_sequences.append(sequence)

        try:
            point_forecast, _ = tfm.forecast(
                inputs=batch_sequences,
                freq=[0] * len(batch_sequences),
            )

            for pred_row in point_forecast:
                batch_predictions.append(pred_row[0])

        except Exception as e:
            print(f"Batch prediction failed at batch {i//batch_size}, using fallback prediction")
            print(f"Error: {str(e)}")

            for j in range(len(batch_X)):
                try:
                    sequence = batch_X[j].flatten()
                    if len(sequence) > 2048:
                        sequence = sequence[-2048:]

                    point_forecast, _ = tfm.forecast(
                        inputs=[sequence],
                        freq=[0],
                    )
                    batch_predictions.append(point_forecast[0][0])
                except:
                    batch_predictions.append(np.mean(batch_X[j]))

        all_predictions.extend(batch_predictions)

        if torch.cuda.is_available() and (i // batch_size) % 10 == 0:
            torch.cuda.empty_cache()
            gc.collect()

    all_predictions = np.array(all_predictions)

    print(f"Prediction completed: {len(all_predictions)} predictions generated")

    permnos_test = meta_test["PERMNO"].values
    metrics = calculate_metrics(y_test, all_predictions, permnos_test, meta_test)

    print("\nDirectional Analysis:")
    print(f"Pos ratio (y_test): {(y_test > 0).mean():.4f}")
    print(f"Neg ratio (y_test): {(y_test < 0).mean():.4f}")
    sign_pred = np.sign(all_predictions)
    print(f"Pred +1 ratio: {(sign_pred > 0).mean():.4f}")
    print(f"Pred -1 ratio: {(sign_pred < 0).mean():.4f}")
    print(f"Pred mean: {np.mean(all_predictions):.6f}")
    print(f"Pred std: {np.std(all_predictions):.6f}")

    results[window_size] = {
        'predictions': all_predictions,
        'true_values': y_test,
        'metrics': metrics,
        'meta': meta_test
    }

    print(f"\nMetrics for window {window_size}:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.6f}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

print("\nTimesFM 2.0 prediction completed for all window sizes!")


Starting TimesFM 2.0 prediction with T4-optimized batch processing...

Processing Window Size: 5
Test data shape: X_test=(110850, 5), y_test=(110850,)
Positive ratio in test: 0.5225
Using batch size: 128 for window size 5


Predicting w5: 100%|██████████| 867/867 [31:48<00:00,  2.20s/it]


Prediction completed: 110850 predictions generated

Directional Analysis:
Pos ratio (y_test): 0.5225
Neg ratio (y_test): 0.4767
Pred +1 ratio: 0.5442
Pred -1 ratio: 0.4558
Pred mean: 0.000483
Pred std: 0.008826

Metrics for window 5:
R²: -0.310588
MAE: 0.013607
MSE: 0.000351
Directional Accuracy: 0.494767
Up_Directional_Acc: 0.536635
Down_Directional_Acc: 0.447433
Top25_R2: -0.313658
Top25_MAE: 0.011760
Top25_MSE: 0.000249
Top25_Dir_Acc: 0.493601
Top25_Up_Acc: 0.489719
Top25_Down_Acc: 0.468775
Bottom25_R2: -0.301648
Bottom25_MAE: 0.017654
Bottom25_MSE: 0.000604
Bottom25_Dir_Acc: 0.492468
Bottom25_Up_Acc: 0.504147
Bottom25_Down_Acc: 0.471041

Processing Window Size: 21
Test data shape: X_test=(110850, 21), y_test=(110850,)
Positive ratio in test: 0.5225
Using batch size: 128 for window size 21


Predicting w21: 100%|██████████| 867/867 [31:49<00:00,  2.20s/it]


Prediction completed: 110850 predictions generated

Directional Analysis:
Pos ratio (y_test): 0.5225
Neg ratio (y_test): 0.4767
Pred +1 ratio: 0.5873
Pred -1 ratio: 0.4127
Pred mean: 0.000657
Pred std: 0.005575

Metrics for window 21:
R²: -0.130615
MAE: 0.012597
MSE: 0.000303
Directional Accuracy: 0.497250
Up_Directional_Acc: 0.579664
Down_Directional_Acc: 0.404316
Top25_R2: -0.125808
Top25_MAE: 0.010852
Top25_MSE: 0.000213
Top25_Dir_Acc: 0.506904
Top25_Up_Acc: 0.596892
Top25_Down_Acc: 0.401353
Bottom25_R2: -0.128659
Bottom25_MAE: 0.016388
Bottom25_MSE: 0.000524
Bottom25_Dir_Acc: 0.505563
Bottom25_Up_Acc: 0.534207
Bottom25_Down_Acc: 0.463806

Processing Window Size: 252
Test data shape: X_test=(110850, 252), y_test=(110850,)
Positive ratio in test: 0.5225
Using batch size: 64 for window size 252


Predicting w252: 100%|██████████| 1733/1733 [33:09<00:00,  1.15s/it]


Prediction completed: 110850 predictions generated

Directional Analysis:
Pos ratio (y_test): 0.5225
Neg ratio (y_test): 0.4767
Pred +1 ratio: 0.6202
Pred -1 ratio: 0.3798
Pred mean: 0.000422
Pred std: 0.002668

Metrics for window 252:
R²: -0.032476
MAE: 0.012012
MSE: 0.000276
Directional Accuracy: 0.502312
Up_Directional_Acc: 0.614678
Down_Directional_Acc: 0.373984
Top25_R2: -0.029877
Top25_MAE: 0.010354
Top25_MSE: 0.000195
Top25_Dir_Acc: 0.431927
Top25_Up_Acc: 0.516848
Top25_Down_Acc: 0.353705
Bottom25_R2: -0.033618
Bottom25_MAE: 0.015636
Bottom25_MSE: 0.000480
Bottom25_Dir_Acc: 0.513556
Bottom25_Up_Acc: 0.525435
Bottom25_Down_Acc: 0.484711

Processing Window Size: 512
Test data shape: X_test=(110850, 512), y_test=(110850,)
Positive ratio in test: 0.5225
Using batch size: 32 for window size 512


Predicting w512: 100%|██████████| 3465/3465 [1:06:20<00:00,  1.15s/it]


Prediction completed: 110850 predictions generated

Directional Analysis:
Pos ratio (y_test): 0.5225
Neg ratio (y_test): 0.4767
Pred +1 ratio: 0.6626
Pred -1 ratio: 0.3374
Pred mean: 0.000600
Pred std: 0.002383

Metrics for window 512:
R²: -0.024708
MAE: 0.011976
MSE: 0.000274
Directional Accuracy: 0.505554
Up_Directional_Acc: 0.658422
Down_Directional_Acc: 0.333050
Top25_R2: -0.021907
Top25_MAE: 0.010321
Top25_MSE: 0.000194
Top25_Dir_Acc: 0.433886
Top25_Up_Acc: 0.655803
Top25_Down_Acc: 0.235974
Bottom25_R2: -0.026431
Bottom25_MAE: 0.015598
Bottom25_MSE: 0.000476
Bottom25_Dir_Acc: 0.515551
Bottom25_Up_Acc: 0.562463
Bottom25_Down_Acc: 0.448538

TimesFM 2.0 prediction completed for all window sizes!


In [ ]:
print("Saving results...")

# Create directories for saving results
os.makedirs("/content/drive/MyDrive/timesfm2.0_results", exist_ok=True)
os.makedirs("/content/drive/MyDrive/timesfm2.0_predictions", exist_ok=True)

# Save the complete results to a pkl file
results_dict = {
    'window_sizes': window_sizes,
    'results': results,
    'model_name': 'TimesFM 2.0',
    'model_version': '2.0',
    'model_params': {
        'backend': 'gpu' if torch.cuda.is_available() else 'cpu',
        'per_core_batch_size': per_core_batch_size,
        'optimal_batch_size': optimal_batch_size,
        'horizon_len': 1,
        'context_len': 2048,
        'num_layers': 50,
        'model_id': 'google/timesfm-2.0-500m-pytorch'
    }
}
joblib.dump(results_dict, "/content/drive/MyDrive/timesfm2.0_results/results.pkl")
print("Complete results have been saved to Google Drive.")

# Save evaluation metrics
metrics_df = pd.DataFrame([
    {**{"Window": window_size}, **results[window_size]["metrics"]}
    for window_size in window_sizes
])
metrics_df.to_csv("/content/drive/MyDrive/timesfm2.0_results/timesfm2.0_metrics.csv", index=False)
print("Metrics have been saved to CSV.")

# Save prediction results
for window_size in window_sizes:
    df = pd.DataFrame({
        "PERMNO": results[window_size]["meta"]["PERMNO"],
        "y_true": results[window_size]["true_values"],
        "y_pred": results[window_size]["predictions"]
    })
    df.to_csv(f"/content/drive/MyDrive/timesfm2.0_predictions/timesfm2.0_w{window_size}.csv", index=False)
    print(f"Predictions for w{window_size} have been saved.")

print("All results have been successfully saved to Google Drive.")


Saving results...
Complete results saved to Google Drive
Metrics saved to CSV
Predictions for w5 saved
Predictions for w21 saved
Predictions for w252 saved
Predictions for w512 saved
All results saved successfully to Google Drive!


In [ ]:
# Display final results summary
print("\n=== TimesFM 2.0 Final Results Summary ===")
print(f"Model: TimesFM 2.0 (500M parameters)")
print(f"Device: {'GPU (T4 optimized)' if torch.cuda.is_available() else 'CPU'}")
print(f"Batch size optimization: {optimal_batch_size}")
print(f"Context length: 2048")

summary_data = []
for window_size in window_sizes:
    metrics = results[window_size]['metrics']
    summary_data.append([
        window_size,
        f"{metrics['R²']:.6f}",
        f"{metrics['MAE']:.6f}",
        f"{metrics['MSE']:.6f}",
        f"{metrics['Directional Accuracy']:.6f}",
        f"{metrics['Up_Directional_Acc']:.6f}",
        f"{metrics['Down_Directional_Acc']:.6f}"
    ])

summary_df = pd.DataFrame(summary_data, columns=[
    'Window Size', 'Adjusted R²', 'MAE', 'MSE',
    'Directional Accuracy', 'Up_Directional_Acc', 'Down_Directional_Acc'
])

print("\nPerformance Summary:")
print(summary_df.to_string(index=False))

summary_df.to_csv("/content/drive/MyDrive/timesfm2.0_results/timesfm2.0_summary.csv", index=False)
print("\nSummary table saved to Google Drive")

print("\nTimesFM 2.0 evaluation completed successfully!")
print("All results have been saved to /content/drive/MyDrive/timesfm2.0_results/")
print("All predictions have been saved to /content/drive/MyDrive/timesfm2.0_predictions/")



=== TimesFM 2.0 Final Results Summary ===
Model: TimesFM 2.0 (500M parameters)
Device: GPU (T4 optimized)
Batch size optimization: 128
Context length: 2048

Performance Summary:
 Window Size Adjusted R²      MAE      MSE Directional Accuracy Up_Directional_Acc Down_Directional_Acc
           5   -0.310588 0.013607 0.000351             0.494767           0.536635             0.447433
          21   -0.130615 0.012597 0.000303             0.497250           0.579664             0.404316
         252   -0.032476 0.012012 0.000276             0.502312           0.614678             0.373984
         512   -0.024708 0.011976 0.000274             0.505554           0.658422             0.333050

Summary table saved to Google Drive

TimesFM 2.0 evaluation completed successfully!
All results have been saved to /content/drive/MyDrive/timesfm2.0_results/
All predictions have been saved to /content/drive/MyDrive/timesfm2.0_predictions/
